In [7]:
import gurobipy as gp
print(gp.gurobi.version())
print(gp.gurobi.licensetype())

(12, 0, 0)


AttributeError: type object 'gurobi' has no attribute 'licensetype'

In [ ]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

#load data
df_songs = pd.read_csv('songs_with_predictions_small.csv')
df_songs_sorted = df_songs.sort_values(by='average_prediction', ascending=False)

target_user = "user_5c0cba28e67a99c235ef2ba9877fef44"
ratings = df_songs[target_user].tolist()
track_names = df_songs['track_name'].tolist()
artists = df_songs['artist_name'].tolist()
unique_artists = list(set(artists))
num_tracks = len(track_names)

#create model
m = Model("Spotify_Base_Model")

#d.v. for each track & artist
x = m.addVars(num_tracks, vtype=GRB.BINARY, name="x")
a = m.addVars(unique_artists, vtype=GRB.BINARY, name="artist")

#constraint for at least 30 songs & link artist variables to selected songs
m.addConstr(quicksum(x[i] for i in range(num_tracks)) >= 30, "min_tracks")
for artist in unique_artists:
    song_indices = [i for i in range(num_tracks) if artists[i] == artist]
    m.addConstr(quicksum(x[i] for i in song_indices) <= 1, f"artist_{artist}")
    m.addConstr(a[artist] == quicksum(x[i] for i in song_indices), f"link_{artist}")

#objective to maximize total predicted rating
m.setObjective(quicksum(ratings[i] * x[i] for i in range(num_tracks)), GRB.MAXIMIZE)
m.optimize()

#extract selected tracks
selected_indices = [i for i in range(num_tracks) if x[i].X > 0.5]
selected_tracks = df_songs.iloc[selected_indices].copy().sort_values(by=target_user, ascending=False)
selected_tracks.head(30)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
print("Total tracks selected:", len(selected_indices))
print("Unique artists selected:", len(set(selected_tracks['artist_name'])))
print("Total predicted score:", selected_tracks[target_user].sum())